# Kaggle Competition

# Table of Contents

1. [Titanic - Machine Learning from Disaster](#ch1)
1. [Deep Dive - Exploratory Data Analysis (EDA)](#ch2)
   1. [Concise summary of the data](#ch2.1)
   1. [Data visualization and analysis](#ch2.2)
1. [Section 3](#section-3)
   - [Subsection 3.1](#subsection-31)
   - [Subsection 3.2](#subsection-32)

<a id="ch1"></a>
# Titanic - Machine Learning from Disaster

![([Titancic Image](#Bibliography))](Images/Titanic.png)


[Titancic Image](#Bibliography)


The sinking of the Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew. While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others ([Cukierski 2012](#Bibliography)). 


#### Dataset Description
| Variable  | Definition                                  | Key                                        |
|-----------|--------------------------                   |--------------------------------------------|
| survival  | Survival                                    | 0 = No, 1 = Yes                            |
| pclass    | Ticket class                                | 1 = 1st, 2 = 2nd, 3 = 3rd                  |
| sex       | Sex                                         |                                            |
| Age       | Age in years                                |                                            |
| sibsp     | # of siblings / spouses aboard the Titanic  |                                            |
| parch     | # of parents / children aboard the Titanic  |                                            |
| ticket    | Ticket number                               |                                            |
| fare      | Passenger fare                              |                                            |
| cabin     | Cabin number                                |                                            |
| embarked  | Port of Embarkation                         | C = Cherbourg, Q = Queenstown, S = Southampton |


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, learning_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score

from termcolor import colored

sns.set_theme(style="whitegrid")

train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

plot_data = train.copy(deep=True)
plot_data["Survived"] = plot_data["Survived"].replace(to_replace={0: "No", 1: "Yes"})


In [ ]:
train.head()

## First Glimpse

How many people survived and did societal norms and priorities play a role during this tragic event?

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
plot_data['Survived'].value_counts().plot.pie(explode=[0,0.1], autopct='%1.1f%%', ax=ax[0, 0])
ax[0, 0].set_title('Survival rate')
ax[0, 0].set_ylabel('')
sns.countplot(data=plot_data, x="Survived", hue="Sex", ax=ax[0, 1])
ax[0, 1].set_title('Survivals by Sex')
sns.histplot(data=plot_data, x="Age", hue="Survived", multiple="stack", bins=20, ax=ax[1, 0])
fig.subplots_adjust(hspace=0.5)
ax[1, 0].set_title('Survivals by Age')
sns.histplot(data=plot_data, x="Fare", hue="Survived", multiple="stack", bins=5, ax=ax[1, 1])
ax[1, 1].set_title('Survivals by Fare')
plt.show()

Clearly, the survival rate of the accident was notably low. Of the 891 passengers in the training set only 350 managed to survive. However it becomes evident that specific attributes like sex, age and socioeconomic status significantly influenced the chances of survival.

Let's conduct a more thorough analysis to uncover valuable insights into the intricate dynamics that determined the fate of the passengers aboard the ill-fated Titanic.




<a id="ch2"></a>
# Deep Dive - Exploratory Data Analysis (EDA)


Exploratory Data Analysis (EDA) is a crucial phase in the data analysis process where the primary goal is to gain insights into the data set, understand its characteristics, and identify patterns or relationships among variables.

### Concise summary of the data

Get a concise summary of the DataFrame including the index, dtype, columns, non-null values and memory usage.

In [ ]:
train.info()

Data formats look reasonable. Looks like we are missing data for Age and Cabin though.

In [ ]:
train.isnull().sum()

Indeed we do and quite so for Cabin (77%) and Age (20%). We will fix this after going over the features.

### Data visualization and analysis

#### Sex

Sex is a categorical feature. Categorical features encompass variables that contain two or more distinct categories. For instance, gender represents the categories 'male' and 'female'. It's important to note that these types of variables cannot be sorted or assigned any specific order. They are alternatively referred to as nominal variables.

In [ ]:
display(pd.crosstab(index=plot_data["Sex"], columns=plot_data["Survived"]).style.set_caption('Survival by Sex'))
display(pd.crosstab(index=plot_data["Sex"], columns=plot_data["Survived"], normalize="index").style.set_caption('Survival rate per sex').\
    format("{:.2f}"))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.countplot(data=plot_data, x="Survived", hue="Sex", ax=ax[0])
ax[0].set_title("Survival by sex")
sns.barplot(data=train[['Sex','Survived']].groupby(['Sex']).mean(), y="Survived", x="Sex", ax=ax[1])
ax[1].set_title("Survival rate by sex")
plt.show()

This is intriguing. Although there were significantly more men aboard the ship compared to women, the quantity of women saved is nearly double that of the men saved. This is also reflected in the survival rate, with women being nearly four times as likely to survive as men. **While women's survival rate on the ship is approximately 75%, men's survival rate is very low with just 19%**. 

The significant disparity can be accounted for by the implementation of the "women first" policy. This policy refers to prioritizing the rescue of women over men during the evacuation. This procedure aimed to protect the most vulnerable and was part of the maritime tradition of prioritizing the safety of women and children in life-threatening situations. 

This raises another question: who were the fortunate men and the unfortunate women in this context? Is there a pattern as well?

#### Passenger Class

Passanger Class is an ordinal feature. Ordinal features are similiar to categorical features, but with an added dimension of relative ordering or ranking between the values. Unlike regular categorical variables, ordinal features have an inherent order among the categories. For instance, if we consider a feature like Education Level with values such as High School, Bachelor's, Master's, and PhD, the feature Education Level would be considered an ordinal variable. In this case, the values can be arranged in a specific order, representing varying levels of education attainment. 


In [ ]:
display(pd.crosstab(index=plot_data["Pclass"], columns=plot_data["Survived"], margins=True).style.set_caption('Survival by Passenger Class'))
display(pd.crosstab(index=plot_data["Pclass"], columns=plot_data["Survived"], margins=True, normalize="index").style.set_caption('Survival rate per Passenger Class').\
    format("{:.2f}"))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.countplot(data=plot_data, x="Pclass", hue="Survived", hue_order= ["No", "Yes"], ax=ax[0])
ax[0].set_title("Survival by Passenger Class")
sns.barplot(data=train[['Pclass','Survived']].groupby(['Pclass']).mean(), y="Survived", x="Pclass", ax=ax[1])
ax[1].set_title("Survival rate by Passanger Class")
plt.show()

People say that money cannot buy everything. However, it is evident that during the rescue efforts, passengers belonging to a higher socioeconomic status were granted significantly higher priority. This is especially reflected in the very low survival rate of passenger in the **third class, with just about 25%**.

But now is that question: did wealth and status also matter in regard of sex? Lets examine the survival rate together with **Sex and Passenger Class**.

In [ ]:
print('Survival by Sex and Passenger Class')
plot_data.groupby(by=["Sex", "Pclass", "Survived"])["Survived"].count()

In [ ]:
grouped_sex_pclass_df  = train.groupby(by=['Sex', 'Pclass']).agg(Survivors = pd.NamedAgg(column='Survived', aggfunc="sum"), Passengers = pd.NamedAgg(column='Survived', aggfunc="size"))
grouped_sex_pclass_df['Survival_Rate'] = grouped_sex_pclass_df['Survivors'] / grouped_sex_pclass_df['Passengers']

print('Survival rate by Sex and Passenger Class')
display(grouped_sex_pclass_df.style.format({'Survival_Rate':'{:.2%}'}))

In [ ]:
fig = sns.catplot(data=train, x="Pclass", y="Survived", col="Sex", kind="bar")

# display mean survival rate for male and female
# choose colors from sns.color_palette().as_hex for mean values
red = (0.7686274509803922, 0.3058823529411765, 0.3215686274509804)
orange = (0.8666666666666667, 0.5176470588235295, 0.3215686274509804)
green = (0.3333333333333333, 0.6588235294117647, 0.40784313725490196)


for ax in fig.axes.flat:
    ax.axhline(y=train['Survived'].mean(), color=orange, linestyle='--', label='Overall Mean')
    if ax.get_title() == 'Sex = male':
        ax.axhline(y=train[train['Sex'] == 'male']['Survived'].mean(), color=red, linestyle='--', label='Male Mean')
    elif ax.get_title() == 'Sex = female':
        ax.axhline(y=train[train['Sex'] == 'female']['Survived'].mean(), color=green, linestyle='--', label='Female Mean')


#create single legend for subplots
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes.flat]
handles, labels = [sum(lol, []) for lol in zip(*lines_labels)]
#specify order of items in legend and omit duplicate label of overall mean
order = [0, 3, 1]
plt.legend(handles=[handles[idx] for idx in order], labels=[labels[idx] for idx in order], title="Survival rate") 


fig.set_axis_labels(x_var="Passenger Class", y_var="Survival rate")
plt.suptitle("Survival Rate by Class and Gender", y=1.02)
plt.show()

 It is evident that irrespective of Passenger class, women were given first priority during the rescue. However money and status were a crucial factor within the groups. The chance of survival virtually doubled for men in Passenger Class 1, coming close to the overall mean. The importance of socioeconomic status becomes even more visible for women, nearly all passangers of the first and second class made it out alive, while in the third class only half the women managed to survive, approaching the overall mean. 
 
 **So women first and rich second**?

#### Age

Age is an continous feature. Continuous features refer to variables that can take on any numerical value within a certain range. In other words, there are no restrictions on the values that a continuous feature can assume between any two points or within the minimum and maximum values in the feature column. Common examples are: temperature, weight, and height. 

In [ ]:
display(train[["Age"]].describe().style.format({'Age':'{:.1f}'}))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.histplot(data=plot_data, x="Age", multiple="layer", bins=20, ax=ax[0])
ax[0].set_title("Age distribution aboard the Titanic")
sns.histplot(data=plot_data, x="Age", hue="Survived", multiple="layer", bins=20, ax=ax[1])
ax[1].set_title("Survival by Age")
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.violinplot(data=plot_data, x="Sex", y="Age", hue="Survived", ax=ax[0])
ax[0].set_title("Distribution of age by gender")
sns.violinplot(data=plot_data, x="Pclass", y="Age", hue="Survived", hue_order= ["No", "Yes"], ax=ax[1])
ax[1].set_title("Distribution of age by passenger class")
plt.show()

Swarmplot gives a better representation of the distribution of values, but it does not scale well to large numbers of observations.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.swarmplot(data=plot_data, x="Sex", y="Age", hue="Survived", size=5, ax=ax[0])
ax[0].set_title("Distribution of age by gender")
sns.swarmplot(data=plot_data, x="Pclass", y="Age", hue="Survived", hue_order= ["No", "Yes"], size=4, ax=ax[1])
ax[1].set_title("Distribution of age by passenger class")
plt.show()

It appears that prioritizing the safety of children, regardless of their gender, was a primary concern — the well-known "women and children first" policy.  However it looks like that while the gender did not matter the status of your parents did. Let's investigate this further.


In [ ]:
df_children = train[train["Age"] <= 16]
children_stats = df_children.groupby(by=['Sex']).agg(Survivors = pd.NamedAgg(column='Survived', aggfunc="sum"),Passengers = pd.NamedAgg(column='Survived', aggfunc="size"), Survival_Rate_children =pd.NamedAgg(column="Survived", aggfunc="mean"))
children_stats["Overall Survival Rate by gender"] = [train[train['Sex'] == 'female']['Survived'].mean(), train[train['Sex'] == 'male']['Survived'].mean()]
children_stats["Overall Survival Rate"] = train['Survived'].mean()
children_stats["Overall Survival Rate children"] = df_children["Survived"].mean()

children_stats
print('Survival rate of children by Sex')
display(children_stats.style.format({'Survival_Rate':'{:.2%}'}))

One can see that the survival rate of children (55%) is clearly higher than the overall average (38%). However while the chance of survival of boys more than doubled (from 19% to 43%), the survival rate of girls is actually slightly lower than the female average (67% against 74%). 

How can that be explained? And which role did the status of your parents play?


In [ ]:
print('Survival rate of children by Sex and Passanger Class')
df_children.groupby(by=['Sex', 'Pclass']).agg(Survivors = pd.NamedAgg(column='Survived', aggfunc="sum"),Passengers = pd.NamedAgg(column='Survived', aggfunc="size"), Survival_Rate =pd.NamedAgg(column="Survived", aggfunc="mean"))

In [ ]:
sns.catplot(data=df_children, x="Pclass", y="Survived", col="Sex", kind="bar")
plt.suptitle("Survival Rate Children by Class and Gender", y=1.02)
plt.show()

It appears that prioritizing the safety of children, regardless of their gender, was a primary concern. However while the gender did not matter the status of your parents did. Almost all children travelling in the first and second passenger class survived whereas children in the third passanger class were not so lucky. Being a child did not matter so much when your parents had low socioeconomic status. Once again, financial status played an important role, even in the context of children's survival.

#### Embarked

Embarked is also a categorical feature. This means there is no ranking between the values.

In [ ]:
print('Survivals by Embarkment and Pclass')
pd.crosstab([plot_data.Embarked, plot_data.Pclass],[plot_data.Sex, plot_data.Survived],margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
fig = sns.catplot(data=train, x="Embarked", y="Survived", kind="bar")
plt.show()

Port C has the highest survival probability at approximately 0.55, whereas S has the lowest. Yet, is this discrepancy attributed to the port itself or other contributing factors?

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(16,12))
sns.countplot(data=plot_data, x='Embarked',ax=ax[0,0])
ax[0,0].set_title('Number of passengers boarded')
sns.countplot(data=plot_data, x='Embarked',hue='Sex',ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot(data=plot_data, x='Embarked',hue='Survived',ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot(data=plot_data, x='Embarked',hue='Pclass',ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2,hspace=0.3)
plt.show()

The majority of passengers boarded from Southampton, primarily from Pclass3. Passengers from Cherbourg experienced a higher survival rate, potentially attributed to the rescue of all Pclass1 and Pclass2 passengers. Despite being popular among wealthy passengers, embarkation at Southampton correlated with lower survival chances, notably with around 81% of Pclass3 passengers not surviving. Port Queenstown had an overwhelming presence of Pclass3 passengers, constituting nearly 95% of the total. 

It kind of seems like Passanger Class and Sex are the decisive factors. Let's examine this:

In [ ]:
grouped_sex_pclass_embarked_df  = train.groupby(by=['Sex', 'Pclass', 'Embarked']).agg(Survivors = pd.NamedAgg(column='Survived', aggfunc="sum"), Passengers = pd.NamedAgg(column='Survived', aggfunc="size"))
grouped_sex_pclass_embarked_df['Survival_Rate'] = grouped_sex_pclass_embarked_df['Survivors'] / grouped_sex_pclass_embarked_df['Passengers']

print('Survival rate by Sex, Passenger Class and Embarked')
display(grouped_sex_pclass_embarked_df.style.format({'Survival_Rate':'{:.2%}'}))

The evaluation reveals the same general trends that we observed before regardless of embarkation location: females exhibit higher survival rates than males and survival rates show a decreasing trend with reduced passenger class.

Nevertheless, one notable observation emerges. The embarkation location appears to influence survival rates within the third passenger class. Approximately 70% of women boarding in Queenstown and Cherbourg survived, contrasting with a lower survival rate of 38% for those embarking in Southampton. One can assume a trend based on embarkation location for males as well.

#### SibSp: Number of siblings / spouses aboard the Titanic 

SibSp represents the number of siblings or spouses aboard the Titanic for each passenger. It is a discrete numerical feature, as it counts the number of individuals, taking on whole, non-negative values. This variable is integral to understanding family structures on the Titanic and can be used to explore the impact of traveling with siblings or spouses.

In [ ]:
print('Survival based on Number of siblings / spouses')
pd.crosstab([plot_data["SibSp"]], plot_data["Survived"])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.countplot(data=plot_data, x="SibSp", hue="Survived", hue_order= ["No", "Yes"], ax=ax[0])
ax[0].set_title("Survival by number of siblings / spouses")
ax[0].set(xlabel='Number of siblings / spouses')
sns.barplot(data=train, x="SibSp", y="Survived", ax=ax[1])
ax[1].set(xlabel='Number of siblings / spouses')
ax[1].set_title("Survival rate by numer of siblings / spouses")
plt.show()

In [ ]:
#TODO: Check and Rewrite this


The barplot and factorplot reveal a noteworthy trend: when passengers are alone without any siblings onboard, their survival rate is 34.5%. The trend appears to decline as the number of siblings increases, suggesting an understandable pattern – individuals with family ties may prioritize saving their relatives over themselves during an emergency. Intriguingly, survival rates for families with 5-8 members drop to 0%. This anomaly prompts consideration of the role of Pclass.

#### Parch: Number of parents / children aboard the Titanic 

Parch is also a discrete numerical feature. Parch stands for the number of parents or children aboard the Titanic for each passenger.

In [ ]:
print('Survival based on number of parents / children')
pd.crosstab([plot_data["Parch"]], plot_data["Survived"])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.countplot(data=plot_data, x="Parch", hue="Survived", hue_order= ["No", "Yes"], ax=ax[0])
ax[0].set_title("Survival by number of parents / children")
ax[0].set(xlabel='Number of parents / children')
sns.barplot(data=train, x="Parch", y="Survived", ax=ax[1])
ax[1].set(xlabel='Number of parents / children')
ax[1].set_title("Survival rate by numer of parents / children")
plt.show()

In [ ]:
#TODO: Check and Rewrite this

Similarly, findings indicate that passengers with parents onboard experience a higher likelihood of survival, although the probability diminishes with an increasing number. Favorable survival odds are observed for those with 1-3 parents onboard. Conversely, traveling alone proves to be precarious, and the probability of survival decreases notably when an individual has more than four parents accompanying them on the ship.

### Observations in a Nutshell 

Gender significantly influences survival chances, with women having notably higher survival rates than men.

Passenger Class (Pclass) reveals a clear pattern: 1st class passengers enjoy better survival prospects, whereas Pclass3 experiences a markedly low survival rate. For women, the likelihood of survival is nearly 100% in Pclass1, and it remains high in Pclass2. Money seems to be a deciding factor.

Age plays a critical role, with children aged 5-10 exhibiting high survival chances. Conversely, passengers between 15 and 35 face elevated mortality rates.

Embarked location adds an intriguing dimension. Despite the majority of Pclass1 passengers embarking at S, the survival rate at C appears better. Notably, passengers at Q are exclusively from Pclass3, correlating with lower survival rates.

#TODO: CHeck this:
Family presence (Parch + SibSp) indicates that having 1-2 siblings or a spouse on board, as well as 1-3 parents, enhances the likelihood of survival. Traveling alone or with a large family diminishes survival probabilities.

## **Feature engineering**

Feature engineering is the process of transforming raw data into a format that is more suitable for machine learning models, with the goal of improving the performance of those models. It involves creating new features, selecting relevant features, and modifying existing features to enhance the model's ability to make accurate predictions or classifications.

### **Impute missing values** 

#### **Age**

Previously, we identified 177 null values in the Age feature and suggested replacing them with the dataset's mean age. However, assigning a universal mean age may not be practical, given the diverse age range. 

A solution lies in examining the Name feature, which includes salutations like Mr or Mrs. We can assign mean values of Mr and Mrs to their respective groups for a more nuanced approach. 
Additionally, we can refine this method by grouping age according to passenger classes. This is beneficial, considering the correlation between age and passenger class.


In [ ]:
print("Number of missing values for Age:", train[["Age"]].isnull().sum())

In [ ]:
# feature engineering create a new feature derived on the initials in name
train["Title"] = train["Name"].str.extract('([A-Za-z]+)\.')

In [ ]:
pd.crosstab(train['Title'], train['Sex']).transpose()

In [ ]:
train.groupby('Title')['Age'].agg(["count", "mean"])


Certain initials, such as Mlle or Mme, most likely represent Miss and have been misspelled. I'll correct these and address similar issues with other values.

In [ ]:
train['Title'] = train['Title'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'])

In [ ]:
train.groupby('Title')['Age'].agg(["count", "mean"])

In [ ]:
train.groupby(by=['Title','Pclass'])['Age'].agg(["count", "mean"])

In [ ]:
missing_age_indices = train[train['Age'].isnull()].index

In [ ]:
train.iloc[missing_age_indices]

In [ ]:
# Fill in missing age values based on title (derived from name) and passenger class
train['Age'] = train['Age'].fillna(train.groupby(by=['Title','Pclass'])['Age'].transform('mean').round(2))

In [ ]:
#Check if imputation was done correctly
train.iloc[missing_age_indices]

In [ ]:
train["Age"].isnull().any()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.histplot(data=train, x="Age", multiple="layer", bins=20, ax=ax[0])
ax[0].set_title("Age distribution aboard the Titanic")
sns.histplot(data=train, x="Age", hue="Survived", multiple="layer", bins=20, ax=ax[1])
ax[1].set_title("Survival by Age")
plt.legend( loc='upper right', labels=['No', 'Yes'])
plt.show()

This looks good and corresponds well to our earlier age histograms.


#### Cabin
`Cabin` feature poses a bit of a challenge and needs further exploration. The mayority of the `Cabin` feature is missing and it cannot be disregarded entirely, as certain cabins may exhibit higher survival rates. The location of the cabin could be interesting for serval reasons:
- Proximity to lifeboats
- Accessibility to escape routes
- Impact location: Depending on the location of the cabin in relation to the point of impact with the iceberg, passengers in certain cabins might have faced different levels of danger and challenges in evacuating the ship.
- Water Ingress:  Cabins at lower levels or closer to the impact point may have experienced quicker and more severe water ingress, impacting the time available for evacuation.


It appears that the initial letter of the Cabin values corresponds to the decks where the cabins are situated. These decks were primarily designated for specific passenger classes, although some were utilized by multiple passenger classes ([Gunesevitan 2020](#Bibliography)).
![alt text](https://vignette.wikia.nocookie.net/titanic/images/f/f9/Titanic_side_plan.png/revision/latest?cb=20180322183733)



Lets' check if people were located differently based on passenger class and embarkment.



In [ ]:
#Feature engineering "Deck" column based on the initial letter of the "Cabin" column, where "M" signifies missing values
train['Deck'] = train['Cabin'].str[0].fillna('M')
plot_data['Deck'] = plot_data['Cabin'].str[0].fillna('M')

In [ ]:
train.groupby(by=["Deck","Pclass"])["Deck"].count()

In [ ]:
train.groupby(by=["Embarked", "Deck"])["Deck"].count()

In [ ]:
sns.countplot(data=plot_data, x="Deck", hue="Embarked")
plt.show()

As so many information for cabins is missing it is hard to tell if people were located on different decks based on the embarkment.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
plot_data.groupby('Deck')['Pclass'].value_counts(normalize=True).unstack('Pclass').plot.bar(stacked=True, ax=ax[0])
ax[0].set_ylabel("Passenger class percentage")
ax[0].set_title("Distribution passenger class by cabin position")
plot_data.groupby('Deck')['Sex'].value_counts(normalize=True).unstack('Sex').plot.bar(stacked=True, ax=ax[1])
ax[1].set_ylabel("Sex percentage")
ax[1].set_title("Distribution sex by cabin position")
plt.suptitle("Distribution of known important factors for Deck", y=1.02)
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
plot_data.groupby('Deck')['Survived'].value_counts(normalize=True).unstack('Survived').plot.bar(stacked=True, ax=ax[0])
ax[0].set_ylabel("Survival Rate")
ax[0].set_title("Survival rate by Cabin position")
sns.countplot(data=plot_data, x="Deck", hue="Survived", ax=ax[1])
ax[1].set_title("Number of survivors by cabin position")
plt.show()


Observations:
* Deck **A**, **B** and **C**  were exclusively for 1st class passengers
* Deck **D** and **E** accomodated passengers of all classes
* Deck **F** and **G** decks were for both 2nd and 3rd class passengers
* From going **A** to **G**, distance to the staircase increases which might be a factor of survival

As suspected, there are distinct survival rate differences among various decks, making this information helpful. Decks B, C, D, and E exhibit the highest survival rates, primarily housing 1st class passengers. Interestingly enough Deck A, though exclusively for 1st class, has a low survival rate. This can be explained due to the high proportion of men and the "women first" policy. Another interesting observation is that Deck C and F have similar survival rates despite having different passenger classes and similar sex distribution. Cabins with no data (Deck M) were mainly occupied by 2nd and 3rd class passengers and have the lowest survival rate. 

With that we can drop Cabin and use Deck as a feature.

It's unclear if Deck is a significant factor itself or a surrogate for passenger class and sex, requiring further investigation into the Titanic sinking process. Some important questions to answer are:
- Where was the boat hit?
- How long did it take for the Titanic to sink?
- Where were the life boats located?
- Had everyone the chance to get on Deck to go to a lifeboat

In [ ]:
missing_cabin_indices = train[train['Cabin'].isnull()].index

In [ ]:
train.iloc[missing_cabin_indices] 

In [ ]:
# Based on our findings we group the passenger cabins into upper, middle and lower deck.
# Mapping for deck categories
deck_mapping = {"A": "Upper", "B": "Upper", "C": "Upper", "D": "Upper",
                "E": "Middle", "F": "Middle", "G": "Lower", "M": "M", "T": "Upper"}

train["Deck"] = train["Deck"].map(deck_mapping)

# Impute missing with upper for 1st Pclass, middle with 2nd Pclass and lower with 3rd Pclass
train.loc[(train["Deck"] =="M") & (train["Pclass"] ==1), "Deck"] = "Upper"
train.loc[(train["Deck"] =="M") & (train["Pclass"] ==2), "Deck"] = "Middle"
train.loc[(train["Deck"] =="M") & (train["Pclass"] ==3), "Deck"] = "Lower"

In [ ]:
#Check if imputation was done correctly
train.iloc[missing_cabin_indices] 

In [ ]:
train["Deck"].isnull().any()

In [ ]:
train["Deck"].value_counts()

#### Embarked

The "Embarked" attribute is a categorical feature, and the entire dataset has only two missing values for this variable. Both instances involve female passengers of the upper class who share identical ticket numbers, suggesting a connection between them. 


In [ ]:
missing_embarked_indices = train[train['Embarked'].isnull()].index

In [ ]:
train.iloc[missing_embarked_indices] 

While the most frequent Embarked value for an upper-class female passenger is C (Cherbourg), it does not conclusively indicate that they boarded the ship from that specific port.

According to ([Gunesevitan 2020](#Bibliography)) the passanger Stone, Mrs. George Nelson (Martha Evelyn) embarked from S (Southampton) with her maid Amelie Icard. 

In [ ]:
train["Embarked"] = train["Embarked"].fillna("S")

### **Binning continous features**


Binning continuous features is a technique used in data analysis and machine learning to categorize continuous variables into discrete bins. This method helps in handling continuous attributes where it is more efficient or meaningful to deal with ranges of values rather than individual values. There are several methods and considerations when it comes to binning continuous features. We will simply split data into equal-sized groups.

#### **Fare**

In [ ]:
train["Fare_Binned"] = pd.qcut(train['Fare'], 13)

In [ ]:
fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Fare_Binned', hue='Survived',  data=train)

plt.xlabel('Fare', size=15, labelpad=20)
plt.ylabel('Passenger Count', size=15, labelpad=20)
plt.tick_params(axis='x', labelsize=10)
plt.tick_params(axis='y', labelsize=15)

plt.legend(['No', 'Yes'], title="Survived", loc='upper right', prop={'size': 15})
plt.title('Survival Count based on Fare', size=15, y=1.05)

plt.show()

In [ ]:
for gender in train['Sex'].unique():
    fig, axs = plt.subplots(figsize=(22, 9))
    sns.countplot(x='Fare_Binned', hue='Survived', data=train[train['Sex'] == gender])

    plt.xlabel('Fare', size=15, labelpad=20)
    plt.ylabel('Passenger Count', size=15, labelpad=20)
    plt.tick_params(axis='x', labelsize=10)
    plt.tick_params(axis='y', labelsize=15)

    plt.legend(['No', 'Yes'], title="Survived", loc='upper right', prop={'size': 15})
    plt.title(f'Survival Count based on Fare for {gender}', size=15, y=1.05)
    plt.show()

Fare, which closely relates to Passenger Class, unsurprisingly yields similar observations: Regardless of the fare amount, women were prioritized during the rescue operation. Nevertheless, within each group, financial means and social standing played a significant role. It's clear that the likelihood of survival noticeably improved for those paying higher fares.

#### **Age**

 Again we will simply split data into equal-sized groups.

In [ ]:
train['Age_Binned'] = pd.qcut(train['Age'], 10)

In [ ]:
fig, axs = plt.subplots(figsize=(22, 9))
sns.countplot(x='Age_Binned', hue='Survived',  data=train)

plt.xlabel('Age', size=15, labelpad=20)
plt.ylabel('Passenger Count', size=15, labelpad=20)
plt.tick_params(axis='x', labelsize=10)
plt.tick_params(axis='y', labelsize=15)

plt.legend(['No', 'Yes'], title="Survived", loc='upper right', prop={'size': 15})
plt.title('Survival Count based on Age', size=15, y=1.05)

plt.show()

In [ ]:
for gender in train['Sex'].unique():
    fig, axs = plt.subplots(figsize=(22, 9))
    sns.countplot(x='Age_Binned', hue='Survived', data=train[train['Sex'] == gender])

    plt.xlabel('Age', size=15, labelpad=20)
    plt.ylabel('Passenger Count', size=15, labelpad=20)
    plt.tick_params(axis='x', labelsize=10)
    plt.tick_params(axis='y', labelsize=15)

    plt.legend(['No', 'Yes'], title="Survived", loc='upper right', prop={'size': 15})
    plt.title(f'Survival Count based on Age for {gender}', size=15, y=1.05)
    plt.show()

As before we see that prioritizing the safety of children and women was a primary concern.


### **Feature Creation**

#### **Family_Size**
The 'Family_Size' attribute is derived by summing the values of 'SibSp', 'Parch'and plus 1 for the current passanger. Here, 'SibSp' represents the number of siblings and spouses aboard, while 'Parch' denotes the number of parents and children present. 

In [ ]:
train['Family_Size'] = train['SibSp'] + train['Parch'] + 1

In [ ]:
sns.countplot(data=train, x="Family_Size", hue="Survived")
plt.legend(['No', 'Yes'], title="Survived", loc='upper right')
plt.show()

In [ ]:
train['Family'] = pd.cut(train['Family_Size'], bins=[0, 1, 4, 6, np.inf], labels=['Alone', 'Small', 'Medium', 'Large'])


In [ ]:
sns.countplot(data=train, x="Family", hue="Survived")
plt.legend(['No', 'Yes'], title="Survived", loc='upper right')
plt.show()

In [ ]:
for gender in train['Sex'].unique():
    fig, axs = plt.subplots(figsize=(22, 9))
    sns.countplot(x='Family', hue='Survived', data=train[train['Sex'] == gender])

    plt.xlabel('Age', size=15, labelpad=20)
    plt.ylabel('Passenger Count', size=15, labelpad=20)
    plt.tick_params(axis='x', labelsize=10)
    plt.tick_params(axis='y', labelsize=15)

    plt.legend(['No', 'Yes'], title="Survived", loc='upper right', prop={'size': 15})
    plt.title(f'Survival Count based on Age for {gender}', size=15, y=1.05)
    plt.show()

#TODO: Write this better:
Family size played a significant role in the Titanic disaster, particularly in the context of evacuation priorities where women and children were given preference. Larger families, especially in third class, faced challenges in accessing lifeboats due to their socioeconomic status and lower deck accommodations. Additionally, the tendency of family members to stay together might have influenced their evacuation decisions, potentially affecting their survival chances.

#### **Ticket**

There are too many unique Ticket values to analyze, so grouping them up by their frequencies makes things easier.

How is this feature different than Family_Size? Many passengers travelled along with groups. Those groups consist of friends, nannies, maids and etc. They weren't counted as family, but they used the same ticket.

Why not grouping tickets by their prefixes? If prefixes in Ticket feature has any meaning, then they are already captured in Pclass or Embarked features because that could be the only logical information which can be derived from the Ticket feature.

In [ ]:
train['Ticket_Frequency'] = train.groupby('Ticket')['Ticket'].transform('count')

In [ ]:
sns.countplot(data=train, x="Ticket_Frequency", hue="Survived")
plt.legend(['No', 'Yes'], title="Survived", loc='upper right')
plt.show()

In [ ]:
train

In [ ]:
train.isnull().any()

In [ ]:
train.corr(numeric_only=True)

## **Data Preprocessing for machine learning models**

Since generally we cannot pass strings to a machine learning models, we need to convert features like Sex, Embarked and so on into numeric values. 

This is more a limitation of the library that we are using than the machine algorithms themselves ([Scikit-learn Documentation Decision Trees](#Bibliography)).

For instance, algorithms like Logistic Regression require all categorical features in a dataset to be converted into a numerically evaluable format. In contrast, Decison Tree models have the inherent capability to process categorical variables without needing them to be numerical. However, in real-world applications, particularly when using scikit-learn, converting these categories to numerical formats is a common practice. This practice is more about aligning with the programming structure of these libraries, rather than being a necessity imposed by the Decison Tree algorithm ([Scikit-learn Documentation Decision Trees](#Bibliography)). This approach ensures broader compatibility and facilitates ease of use across various machine learning scenarios.


The following block is for preparing the dataset. The following steps will be carried out:

1) Feature selection
2) Encoding of nominal categorical features: Apply One-Hot Encoding to transform nominal categorical features (those without any intrinsic order) into a binary format
3) Encoding of ordinal features: Transform ordinal categorical features, which possess a logical order or ranking, into binary format by utilizing Label Encoding
4) Preparation of training and test sets


In [ ]:
# Dropping unnecessary features
# drop_cols = ["PassengerId", "Name", 'Age', 'Ticket', 'Fare', 'Cabin', 'Ticket_Frequency', "Title", "Deck", "Fare_Binned", "Family_Size", "SibSp", "Parch"]
drop_cols = ["PassengerId", "Name", 'Age', 'Ticket', 'Fare', 'Cabin']
train_processed = train.drop(columns=drop_cols , axis=1)


#Adjusting the encoding
## One-Hot Encoding
# cols_one_hot_encoding = ["Sex", "Embarked", "Title"]
cols_one_hot_encoding = ["Embarked", "Title"]

train_processed = pd.get_dummies(train_processed, columns=cols_one_hot_encoding, drop_first=True)

##Label encoding
label_encoder = LabelEncoder()

cols_label_encoding = ["Sex","Deck", "Fare_Binned", "Age_Binned", "Family"]
#cols_label_encoding = ["Sex", "Age_Binned", "Family"]
for feature in cols_label_encoding:
    train_processed[feature] = label_encoder.fit_transform(train_processed[feature])
    le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print('Mapping target->label', le_name_mapping)


#Splitting the data into training and test data
X, y = train_processed.drop(columns=["Survived"], axis=1), train_processed[["Survived"]] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nControl the shapes of the training and test sets")
print('Shape X:', X.shape)
print('Shape y:', y.shape)
print('Shape  X_train:', X_train.shape)
print('Shape  X_test:', X_test.shape)
print('Shape  y_train:', y_train.shape)
print('Shape  y_test:', y_test.shape)

In [ ]:
train_processed.corr()

## **Model**

Having navigated through exploratory data analysis and feature engineering, we now enter the predictive modeling stage where we transform our hypotheses and insights into actionable predictions. Our first focus is the Decision Tree algorithm, where we'll explore its workings in detail. This will set the foundation for efficiently applying and evaluating other algorithms. Let's dive in and see how well our data can predict outcomes!


The following is a list of algorithms that will be utilized to develop the predictive model:

- Decision tree
- Logistic Regression
- SVC classifier
- Ensemble random forest
- Gradient boosting machine
- Light GBM
- XgBoost Classifier


Model outputs are stored in 2 tables:

- model_params - This stores the model name and relevant score metrics
- Model prediction profile (model_predictions) - This stores the test set predictions from each model (best estimator)

### **Decision Tree**

#### **First Model**

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X=X_train, y=y_train)

y_pred = clf.predict(X_test)

#Different metrics
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
precision = precision_score(y_true=y_test, y_pred=y_pred)
recall = recall_score(y_true=y_test, y_pred=y_pred)
f1 = f1_score(y_true=y_test, y_pred=y_pred)
roc_auc = roc_auc_score(y_true=y_test, y_score=y_pred)

metrics = [f'Accuracy for Decision Tree is {accuracy}', 
           f'Precision for Decision Tree is {precision}',
           f'Recall for Decision Tree is {recall}',
           f'F1 for Decision Tree is {f1}',
           f'ROC/AUC for Decision Tree is {roc_auc}'
           ]

for metric in metrics:
    print(metric)

#### **Overfitting**

Overfitting occurs when a machine learning model captures not only the underlying patterns in the training data but also the noise or random fluctuations specific to that dataset. As a result, while the model performs exceptionally well on the training data, its ability to generalize to new, unseen data is significantly impaired, leading to poor performance on test or real-world data.

Decision Trees are prone to overfitting, especially when they are allowed to grow without constraints. Let's check how our model is doing.


In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X=X_train, y=y_train)

y_pred = clf.predict(X_train)

#Different metrics
accuracy = accuracy_score(y_true=y_train, y_pred=y_pred)
precision = precision_score(y_true=y_train, y_pred=y_pred)
recall = recall_score(y_true=y_train, y_pred=y_pred)
f1 = f1_score(y_true=y_train, y_pred=y_pred)
roc_auc = roc_auc_score(y_true=y_train, y_score=y_pred)

metrics = [f'Accuracy for Decision Tree is {accuracy}', 
           f'Precision for Decision Tree is {precision}',
           f'Recall for Decision Tree is {recall}',
           f'F1 for Decision Tree is {f1}',
           f'ROC/AUC for Decision Tree is {roc_auc}'
           ]

for metric in metrics:
    print(metric)

In [ ]:
# Calculate learning curves
train_sizes, train_scores, test_scores = learning_curve(clf, X, y, cv=5, n_jobs=-1, 
                                                        train_sizes=np.linspace(0.1, 1.0, 10))

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)


test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label="Training score",  marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15)

plt.plot(train_sizes, test_mean, label="Test score", marker='o')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std,  alpha=0.15)

plt.title(f"Learning Curve for Decision Tree")
plt.xlabel("Training Data Size")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

We see signs of overfitting for both in the performance gap in our metrics as well as in the learning curve.

#### **Cross Validation**

Cross-validation is a statistical technique used in machine learning to assess the predictive performance of models and mitigate overfitting. It involves splitting the data into multiple segments, enabling comprehensive model evaluation, hyperparameter tuning, and ensuring model robustness across different subsets of data.



In [ ]:
cv_result = cross_val_score(estimator=clf, X=X_test ,y=y_test, cv=5)

print(f'Accuracy of the five-fold cross-validation: {cv_result}\n Mean: {cv_result.mean()}\n Standard Deviation: {cv_result.std()}')

#### **Hyperparameter Tuning using GridSearch**

GridSearchCV is a technique used in machine learning for tuning hyperparameters, the parameters that are not directly learnt within estimators. It works by systematically working through multiple combinations of parameter tunes, cross-validating as it goes to determine which tune gives the best performance.

In [ ]:
clf = DecisionTreeClassifier()
params = {'max_depth': range(4,8,1), 'min_samples_leaf' : range(3,12,1)}

clf_GCV = GridSearchCV(estimator=clf, param_grid=params, cv=2, scoring="accuracy", refit=True)

clf_GCV.fit(X_train, y_train)

print('Best paramaters are:',clf_GCV.best_params_)

#Plot accuracy for the different hyperparameter combinations
multi_index = pd.MultiIndex.from_product([params['max_depth'], params['min_samples_leaf']], 
                                         names=['max_depth', 'min_samples_leaf'])

# Creating a DataFrame with the MultiIndex
hyperparameter_grid= pd.DataFrame(index=multi_index)
hyperparameter_grid["mean_test_score"] = clf_GCV.cv_results_["mean_test_score"]
hyperparameter_grid["std_test_score"] = clf_GCV.cv_results_['std_test_score']


plt.figure(figsize=(10, 6))
sns.lineplot(data=hyperparameter_grid, x='min_samples_leaf', y='mean_test_score', hue='max_depth', marker='o')

plt.xlabel('Min Samples Leaf')
plt.ylabel('Mean Test Score')
plt.title('Mean Test Score by Max Depth and Min Samples Leaf', y=1.02)
plt.legend(title='Max Depth')
plt.grid(True)
plt.show()

#### **Visual representation of the decision tree**
The funciton 'plot_tree' helps to visually represent a decision tree model. It graphically displays the structure of the decision tree, showing the nodes, branches, and leaves, as well as the decisions or splits made at each node. This visualization helps in understanding how the decision tree model makes predictions based on the features of the input data.

In [ ]:
# Tree with the best parameters
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)
tree = tree.fit(X=X_train, y=y_train)

y_pred = tree.predict(X_test)

plt.figure(figsize=(12, 8))
plot_tree(tree, max_depth=2, feature_names=X_train.columns, label='all', filled=True)
plt.show()

#### **Feature Importance**
Feature importance refers to techniques that assign a score to input features based on how useful they are at predicting a target variable. Tree-based models like Decision Trees support feature importance.

In [ ]:
feature_importance = pd.DataFrame(data={'Feature': X_train.columns, 'Importance': tree.feature_importances_}).sort_values(by="Importance", ascending=False)
sns.barplot(data=feature_importance, x="Importance", y="Feature", hue="Feature")
plt.ylabel("")
plt.title('Decision Tree feature importance', size=15, y=1.02)
plt.show()

#### **Confusion Matrix**
A confusion matrix is a tool used in classification tasks to visualize the performance of a machine learning model. It is a table with four elements showing the number of true positives, false positives, true negatives, and false negatives. This matrix helps in assessing the model's accuracy, identifying the types of errors it makes, and informing further improvements.

In [ ]:
from sklearn.metrics import confusion_matrix

sns.heatmap(confusion_matrix(y_true=y_test, y_pred=y_pred),  annot=True, fmt='g', cmap='Blues', cbar=False)

plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for Decision Tree', y=1.02)

axis_labels = ['No', 'Yes']  
plt.xticks([0.5, 1.5], axis_labels)
plt.yticks([0.5, 1.5], axis_labels)
plt.show()

### **Various classification algorithms**
**Implementation Procedure**

1. Load the appropriate model
2. Apply the grid search parameters to tune the model
3. Generate scores for the training dataset
4. Derive predictions for the test dataset.
5. Incorporate the results into the designated output tables.


In [ ]:
random_state = 10
# Creating a dictionary for the model parameters and classifier class instances
model_param_dict = \
{
'Logistic': [LogisticRegression(random_state=random_state, max_iter=400), {'C': np.exp(np.linspace(-2,2,5))}],
'SVC': [SVC(random_state =random_state), {'C': range(3,10,1)}],
'DTree' : [DecisionTreeClassifier(random_state=random_state), {'max_depth': range(2,7,1), 'min_samples_leaf' : range(3,12,1), 'max_leaf_nodes': range(2,12,1)}],
# 'RandomForest': [RandomForestClassifier(random_state =random_state), {'n_estimators': range(50,300,25), 'max_depth': range(4,7,1)}],
# 'GBM': [GradientBoostingClassifier(random_state=random_state), {'max_depth' : range(2,6,1)}],
# 'LGBM': [LGBMClassifier(random_state =random_state, importance_type="gain", verbosity=-1),{}],
# 'XgBoost': [XGBClassifier(eval_metric= 'logloss'), {}]
}

cv = None

# Creating empty model output storage objects
model_parameters = pd.DataFrame(data= None, index= list(model_param_dict.keys()), dtype= np.float32,
                             columns= ['Precision_Score', 'Recall_Score', 'F1_Score', 'ROC_AUC_Score', 'Accuracy_Score'])
model_parameters_test = pd.DataFrame(data= None, index= list(model_param_dict.keys()), dtype= np.float32,
                             columns= ['Precision_Score', 'Recall_Score', 'F1_Score', 'ROC_AUC_Score', 'Accuracy_Score'])
model_predictions = pd.DataFrame(data= None, index= None, columns= None, dtype= np.int32)

In [ ]:
supports_feature_importance = ["DTree", "RandomForest", "GBM", "LGBM", "XgBoost"]

for model, model_params in model_param_dict.items():
    print(model_params[0], model_params[1])
    print(colored(f"\nCurrent model is {model}", color='red', attrs=['bold', 'dark']))
    grid = GridSearchCV(estimator=model_params[0], param_grid=model_params[1], scoring='accuracy', refit=True, cv=cv)
    grid.fit(X_train, y_train["Survived"].values)
    print(f"""Best estimator is \n{grid.best_estimator_}\n""")
    
    y_train_pred = grid.predict(X_train)
    y_test_pred = grid.predict(X_test)
   
   
    #Display confusion matrix
    sns.heatmap(confusion_matrix(y_true=y_train['Survived'].values, y_pred=y_train_pred),  annot=True, fmt='g', cmap='Blues', cbar=False)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion Matrix for {model}', y=1.02)
    axis_labels = ['No', 'Yes']  
    plt.xticks([0.5, 1.5], axis_labels)
    plt.yticks([0.5, 1.5], axis_labels)
    plt.show()

    #Display feature importance
    if model in supports_feature_importance:
        best_model = grid.best_estimator_
        best_model.fit(X_train, y_train["Survived"].values)

        # for LGBM feature importance is not automatically normalized
        if model == "LGBM":
            feature_importance = pd.DataFrame(data={'Feature': X_train.columns, 'Importance': best_model.feature_importances_ / sum(best_model.feature_importances_)}).sort_values(by="Importance", ascending=False)
        else:
            feature_importance = pd.DataFrame(data={'Feature': X_train.columns, 'Importance': best_model.feature_importances_}).sort_values(by="Importance", ascending=False)
                  
        sns.barplot(data=feature_importance, x="Importance", y="Feature", hue="Feature")
        plt.ylabel("")
        plt.title(f'Feature importance of {model}', size=15, y=1.02)
        plt.show()

    
    # Display learning curve
    train_sizes, train_scores, test_scores = learning_curve(grid.best_estimator_, X, y, cv=5, n_jobs=-1, 
                                                            train_sizes=np.linspace(0.1, 1.0, 10))

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)


    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, label="Training score",  marker='o')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15)

    plt.plot(train_sizes, test_mean, label="Test score", marker='o')
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std,  alpha=0.15)

    plt.title(f"Learning Curve for {model}")
    plt.xlabel("Training Data Size")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    model_parameters.loc[model] = (precision_score(y_true=y_train['Survived'].values, y_pred=y_train_pred),
                                  recall_score(y_true=y_train['Survived'].values, y_pred=y_train_pred),
                                  f1_score(y_true=y_train['Survived'].values, y_pred=y_train_pred),
                                  roc_auc_score(y_true=y_train['Survived'].values, y_score=y_train_pred),
                                  accuracy_score(y_true=y_train['Survived'].values, y_pred=y_train_pred)
                                 )
    
    model_parameters_test.loc[model] = (precision_score(y_true=y_test['Survived'].values, y_pred=y_test_pred),
                                  recall_score(y_true=y_test['Survived'].values, y_pred=y_test_pred),
                                  f1_score(y_true=y_test['Survived'].values, y_pred=y_test_pred),
                                  roc_auc_score(y_true=y_test['Survived'].values, y_score=y_test_pred),
                                  accuracy_score(y_true=y_test['Survived'].values, y_pred=y_test_pred)
                                 )
    
    model_predictions = pd.concat((model_predictions, pd.DataFrame(data=grid.predict(X_test), index=X_test.index, columns =[model], dtype=np.int32)),
                             axis=1, join='outer')
print(colored(f"\n\nTraining set scores across models:\n", color = 'blue', attrs= ['bold', 'dark']))
display(model_parameters.style.format('{:.2%}'))
print(colored(f"\n\nTest set scores across models:\n", color = 'blue', attrs= ['bold', 'dark']))
display(model_parameters_test.style.format('{:.2%}'))

# Bibliography

Cukierski, Will (2012). "Titanic - Machine Learning from Disaster". Publisher: Kaggle. URL: [https://kaggle.com/competitions/titanic](https://kaggle.com/competitions/titanic).

Gunesevitan, "Titanic: Advanced Feature Engineering Tutorial", Kaggle, available at: [https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial](https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial) (accessed on [02.01.2024]).

Scikit-learn Documentation Decision Trees, "Decision Trees — scikit-learn 1.3.2 documentation", available at: [https://scikit-learn.org/stable/modules/tree.html#tree](https://scikit-learn.org/stable/modules/tree.html#tree) (accessed on 05.01.2024).

Titancic Image, (2024). "Sinking of the RMS Titanic Titanic". Publisher: cleanpng.com. URL: [https://www.cleanpng.com/png-sinking-of-the-rms-titanic-titanic-honor-and-glory-2205845/download-png.html](https://www.cleanpng.com/png-sinking-of-the-rms-titanic-titanic-honor-and-glory-2205845/download-png.html).


**References**

This notebook has been created based on great work done solving the Titanic competition and other sources.

- A journey through Titanic
- Getting Started with Pandas: Kaggle's Titanic Competition
- Titanic Best Working Classifier